In [4]:
import os
import cv2
import dlib
import numpy as np

In [5]:
dataset_path = 'Dataset_1'
X = []
y = []
for subject_name in os.listdir(dataset_path):

    y.append(subject_name)
    subject_images_dir = os.path.join(dataset_path, subject_name)

    temp_x_list = []
    for img_name in os.listdir(subject_images_dir):
        # write code to read each 'img'
        image_path = os.path.join(subject_images_dir, img_name)
        img = cv2.imread(image_path)  # the shape is (480, 640, 3) -> print(img.shape)

        # img to temp_x_list add the
        if img is not None:
            temp_x_list.append(img)
    X.append(temp_x_list)  # list type, element is numpy, (50, 15, 480, 640, 3)

In [6]:
nameIndex=0
name = 0
X_processed = []
detector = dlib.get_frontal_face_detector()
for x_list in X:  # x_list: (15, 480, 640, 3)
    temp_X_processed = []
    name = name + 1
    folder_name = "./unmaskedSet/"+str(name)
    folder = os.path.exists(folder_name)
    if not folder:
        os.makedirs(folder_name)
    nameIndex = 0
    for x in x_list:  # x: (480. 640, 3)
        # convert the image (x) to grayscale
        gray_img = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
        faces = detector(gray_img, 1)  # actually only one face in our dataset
        # crop the image (x) to keep only the face, resize the cropped image to 150x150
        left, top, right, bottom = 0, 0, 0, 0
        for _, j in enumerate(faces):
            left = j.left()
            top = j.top()
            right = j.right()
            bottom = j.bottom()

        img_crop = gray_img[top:bottom, left: right]  # crop
        img_reshape = cv2.resize(img_crop, (150, 150))  # resize to (150, 150), type is numpy
        picture_path = folder_name+"/"+str(nameIndex)+'.png'
        nameIndex+=1
        cv2.imwrite(picture_path,img_reshape)
        # append the converted image into temp_X_processed
        temp_X_processed.append(img_reshape)
    # append temp_X_processed into  X_processed
    X_processed.append(temp_X_processed)  # list type, element is numpy, (50, 15, 150, 150), faces

In [7]:
X_masked = []
nameIndex = 0
name = 0
for x_list in X_processed:
    temp_X_masked = []
    name = name + 1
    folder_name = "./maskSet/"+str(name)
    folder = os.path.exists(folder_name)
    if not folder:
        os.makedirs(folder_name)
    nameIndex = 0
    for x in x_list:
        # detect face in the image (x) using dlib facedetection library
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
        faces = detector(x, 1)
        for face in faces:
            landmarks = predictor(x, face)
        # add synthetic mask
        points = []
        for i in range(1, 16):
            point = [landmarks.part(i).x, landmarks.part(i).y]
            points.append(point)

        mask_a = [[landmarks.part(42).x, landmarks.part(15).y],
                  [landmarks.part(27).x, landmarks.part(27).y],
                  [landmarks.part(39).x, landmarks.part(1).y]]
        fin_mask = points + mask_a
        fin_mask = np.array([fin_mask], dtype=np.int32)

        img2 = cv2.polylines(x, fin_mask, True, 1, thickness=2, lineType=cv2.LINE_8)
        img3 = cv2.fillPoly(img2, fin_mask, 1, lineType=cv2.LINE_AA)
        # append the converted image into temp_X_masked
        temp_X_masked.append(img3)
        picture_path = folder_name+"/"+str(nameIndex)+'.png'
        nameIndex+=1
        cv2.imwrite(picture_path,img3)
    # append temp_X_masked into  X_masked
    X_masked.append(temp_X_masked)  # list type, element is numpy, (50, 15, 150, 150), face with mask